# Displaying Status Information

## Overview

### Questions

* What is a Logger? 
* How do I display status information during a simulation?

### Objectives

* Describe and give examples of **loggable quantities**.
* Show how to add quantities to a **Logger**.
* Demonstrate **user-defined log quantities**.
* Explain the use of **Table** to display status information during a simulation run.

## Boilerplate code

In [1]:
import hoomd
import datetime

## Components of the Logging system

There are three parts to the logging system in HOOMD-blue: **Loggable quantities**, the **Logger** class, and 
logging backends.
**Loggable quantities** are values computed during a simulation.
The **Logger** class provides a way to collect and name quantities of interest.
Backends write these values out in a format you can use.
This section will demonstrate the **Table** backend to display status information during a simulation run.

## Define the Simulation

This tutorial executes the Lennard-Jones particle simulation from a previous tutorial. 
See *Introduction to Molecular Dyamics* for a complete description of this code.

In [2]:
gpu = hoomd.device.GPU()
sim = hoomd.Simulation(device=gpu)
sim.create_state_from_gsd(filename='../01-Introducing-Molecular-Dynamics/random.gsd')

integrator = hoomd.md.Integrator(dt=0.005)
cell = hoomd.md.nlist.Cell()
lj = hoomd.md.pair.LJ(nlist=cell)
lj.params[('A', 'A')] = dict(epsilon=1, sigma=1)
lj.r_cut[('A', 'A')] = 2.5
integrator.forces.append(lj)
nvt = hoomd.md.methods.NVT(kT=1.5, filter=hoomd.filter.All(), tau=1.0)
integrator.methods.append(nvt)
sim.operations.integrator = integrator

## Loggable quantities

Many classes in HOOMD-blue provide special *properties* called **loggable quantities**.
For example, the **Simulation** class provides `timestep`, `tps`, and others.
The reference documentation labels each of these as `Loggable`.
You can also examine the ``loggables`` property to determine the **loggable quantities**:

In [3]:
sim.loggables

{'timestep': 'scalar',
 'tps': 'scalar',
 'walltime': 'scalar',
 'final_timestep': 'scalar'}

**Loggable quantities** are class properties, so you can directly access them in your code.

In [4]:
sim.timestep

30000

Each **loggable quantity** has a **flag**, which is listed both in the reference documentation and in ``loggables``.
The **flag** is a string that indicates the quantity's type.
Examples of flags include `scalar`, `string`, and `particle` which describe scalar numbers, strings of characters, and arrays of per-particle numbers or vectors.

## Create a Logger

Add each of the quantities you would like to see to a **Logger**.
When constructing the **Logger**, you can define which **flags** it will accept.
**Table**, which you will use later in this section, only accepts `scalar` and `string` flags:

In [5]:
logger = hoomd.logging.Logger(flags=['scalar', 'string'])

Add the `timestep` and `tps` quantities from `sim` to the logger:

In [6]:
logger.add(sim, quantities=['timestep', 'tps'])

[('Simulation', 'timestep'), ('Simulation', 'tps')]

You can add **loggable quantities** from any number of objects to a **Logger**.
**Logger** uses the namespace of the class to assign a unique name for each quantity.

You can also set user-defined quantities using functions or callable class instances.
For example, this function computes the time remaining in the run:

In [7]:
def eta():
    seconds_remaining = (sim.final_timestep - sim.timestep) / sim.tps
    return str(datetime.timedelta(seconds=seconds_remaining))

logger[('ETA')] = (eta, 'string')

## Display quantities with Table

**Table** is a **Writer** that formats the quantities in a **Logger** into a human readable table.
Create one that triggers periodically:

In [8]:
table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(period=5000),
                          logger=logger)

Add it to the simulation:

In [9]:
sim.operations.writers.append(table)

Run the simulation and see the output:

In [10]:
sim.run(100000)

Simulation.timestep  Simulation.tps        ETA       
       35000           5189.44754     0:00:18.306380 
       40000           6335.27339     0:00:14.206175 
       45000           7834.54429     0:00:10.849387 
       50000           8890.23830     0:00:08.998634 
       55000           9671.93795     0:00:07.754392 
       60000          10273.39441     0:00:06.813717 
       65000          10750.55451     0:00:06.046200 
       70000          11139.75027     0:00:05.386117 
       75000          11461.89828     0:00:04.798507 
       80000          11733.40605     0:00:04.261337 
       85000          11965.57246     0:00:03.760790 
       90000          12164.80833     0:00:03.288173 
       95000          12341.07067     0:00:02.836059 
      100000          12495.90307     0:00:02.400787 
      105000          12632.65518     0:00:01.978998 
      110000          12754.73517     0:00:01.568045 
      115000          12865.27402     0:00:01.165929 
      120000          12963.

In this section, you have displayed **loggable quantities** during a simulation run. You might find this useful for monitoring the status of long-running jobs. The remaining sections in this tutorial will show you how to save these values to a file, log quantities from other objects, and log per-particle quantities along with a trajectory.

[Index](00-index.ipynb) / [Next section](02-Writing-to-a-File.ipynb).